In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn import set_config; set_config(display='diagram')

In [2]:
key = os.environ.get('API_KEY')

# Get Data

In [3]:
# Step 1: Define a function to read CSV files and convert them to dataframes
# Get the current working directory (where your script is located)
current_directory = os.getcwd()

# Specify the path to the "raw_data" folder
raw_data_folder = os.path.join(current_directory, "raw_data")

# List all CSV files in the "raw_data" folder
csv_files = [os.path.join(raw_data_folder, file) for file in os.listdir(raw_data_folder) if file.endswith(".csv")]

# passing csv files in to a dataframe
dataframes = [pd.read_csv(file) for file in csv_files]

In [4]:
# Step 2: Define a function to read CSV files and convert them to dataframes
def read_csv_to_dataframe(file_path):
    df = pd.read_csv(file_path)
    return df

In [5]:
# Step 4: Read CSV files, preprocess data, and stack them into a 3D tensor
data = []
stock_names =[]

# print(type(csv_files))

for file in csv_files:
    # Read the CSV file into a DataFrame
    data_df = pd.read_csv(file)
    stock_name = file.split('/')[-1].split('.')[0]
    stock_names.append(stock_name)
    data_df["stock"] = stock_name
    data.append(data_df)

# Pipelines

In [6]:
impute_columns = ["sma25", "sma100", "sma200", "rsi", "macd", "signal", "histogram"]

imputer = Pipeline(
    [
        ('imputer', ColumnTransformer(
            transformers=[
                ('impute', KNNImputer(n_neighbors=10), impute_columns),  # Apply imputation to specific columns
            ],
            remainder='passthrough'  # Keep the remaining columns
        ))
    ]
)


imputer

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute',
                                                  KNNImputer(n_neighbors=10),
                                                  ['sma25', 'sma100', 'sma200',
                                                   'rsi', 'macd', 'signal',
                                                   'histogram'])]))])

In [7]:
def df_and_column_transform(arr):
    df = pd.DataFrame(arr, columns=["sma25", "sma100", "sma200", "rsi", "macd", "signal", "histogram",\
    "Unnamed: 0", "open", "high","low", "close", "volume", "vwap", "timestamp", "transactions", "otc", "stock"])
    
    first_col = df.pop("sma25")
    df.insert(16, "sma25", first_col)

    sec_col = df.pop("sma100")
    df.insert(16, "sma100", sec_col)

    third_col = df.pop("sma200")
    df.insert(16, "sma200", third_col)

    fourth_col = df.pop("rsi")
    df.insert(16, "rsi", fourth_col)

    fifth_col = df.pop("macd")
    df.insert(16, "macd", fifth_col)
    
    sixth_col = df.pop("signal")
    df.insert(16, "signal", sixth_col)

    seventh_col = df.pop("histogram")
    df.insert(16, "histogram", seventh_col)
    
    return df

transform_pipe = make_pipeline(
    FunctionTransformer(df_and_column_transform)
)

preprocessor = preprocessor = Pipeline(
    [
        ("transformation", transform_pipe),
    ],
)
preprocessor


def timestamp_transform(df):
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    first_col = df.pop("date")
    df.insert(0, 'date', first_col)
    # df.set_index(keys='date', inplace=True)
    
    return df

time_pipe = make_pipeline(
    FunctionTransformer(timestamp_transform)
)

preprocessor = Pipeline(
    [
        ("transformation", transform_pipe),
        ("timestamp_convertor", time_pipe),
    ],
)
preprocessor

drop_columns = ["Unnamed: 0", 'timestamp', "transactions", "otc"]

def drop(df):
    unwanted_columns = drop_columns
    df = df.drop(columns=unwanted_columns)
    
    return df

drop_pipe = make_pipeline(
    FunctionTransformer(drop)
)

preprocessor = Pipeline(
    [
        ("transformation", transform_pipe),
        ("timestamp_convertor", time_pipe),
        ("unwanted_columns", drop_pipe),
    ],
)
preprocessor

Pipeline(steps=[('transformation',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function df_and_column_transform at 0x7fdd24678550>))])),
                ('timestamp_convertor',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function timestamp_transform at 0x7fdd24678670>))])),
                ('unwanted_columns',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function drop at 0x7fdd24678700>))]))])

In [8]:
sec_pipe = Pipeline(
    [
        ("imputer", imputer),
        ("preprocessor", preprocessor),
    ]
)

sec_pipe

Pipeline(steps=[('imputer',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('impute',
                                                                   KNNImputer(n_neighbors=10),
                                                                   ['sma25',
                                                                    'sma100',
                                                                    'sma200',
                                                                    'rsi',
                                                                    'macd',
                                                                    'signal',
                                                                    'histogram'])]))])),
                ('preprocessor',
                 Pipeline(steps=[('transformation',
                                  Pipeline(steps=[('functiontransformer',
                                                   FunctionTransformer(func=<function df_and_column_transform at 0x7fdd24678550>))])),
                                 ('timestamp_convertor',
                                  Pipeline(steps=[('functiontransformer',
                                                   FunctionTransformer(func=<function timestamp_transform at 0x7fdd24678670>))])),
                                 ('unwanted_columns',
                                  Pipeline(steps=[('functiontransformer',
                                                   FunctionTransformer(func=<function drop at 0x7fdd24678700>))]))]))])

In [9]:
scaler_columns = ["open", "high", "low", "close", "volume", "vwap", "sma25", "sma100",
       "sma200", "rsi", "macd", "signal", "histogram"]

exclude = ["stock"]

# Create transformers for numeric and object columns
numeric_transformer = Pipeline(
    steps=[
        ('scaler', MinMaxScaler())
    ]
)

object_transformer = Pipeline(steps=[])

# Combine transformers using ColumnTransformer
sec_scaler = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, scaler_columns),
        # ('obj', object_transformer, exclude),
    ],
    remainder='passthrough'  # Keep the remaining columns
)

# Create a final pipeline
scaler = Pipeline(
    steps=[
        ('scaler', sec_scaler)
    ]
)


scaler

Pipeline(steps=[('scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  ['open', 'high', 'low',
                                                   'close', 'volume', 'vwap',
                                                   'sma25', 'sma100', 'sma200',
                                                   'rsi', 'macd', 'signal',
                                                   'histogram'])]))])

In [10]:
penul_pipe = Pipeline(
    [
        ("imputer", imputer),
        ("preprocessor", preprocessor),
        ("scaler", scaler)
    ]
)

penul_pipe

Pipeline(steps=[('imputer',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('impute',
                                                                   KNNImputer(n_neighbors=10),
                                                                   ['sma25',
                                                                    'sma100',
                                                                    'sma200',
                                                                    'rsi',
                                                                    'macd',
                                                                    'signal',
                                                                    'histogram'])]))])),
                ('preprocessor',
                 Pipeline(steps=[('transformation',
                                  Pipeline(steps=[('functiontransformer',
                                                   FunctionTransformer(func=<function df_and_colu...
                                  Pipeline(steps=[('functiontransformer',
                                                   FunctionTransformer(func=<function drop at 0x7fdd24678700>))]))])),
                ('scaler',
                 Pipeline(steps=[('scaler',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    MinMaxScaler())]),
                                                                   ['open',
                                                                    'high',
                                                                    'low',
                                                                    'close',
                                                                    'volume',
                                                                    'vwap',
                                                                    'sma25',
                                                                    'sma100',
                                                                    'sma200',
                                                                    'rsi',
                                                                    'macd',
                                                                    'signal',
                                                                    'histogram'])]))]))])

In [11]:
def final_transformation(arr):
    cols = ["open", "high", "low", "close", "volume", "vwap", "sma25", "sma100", "sma200", "rsi", "macd",\
        "signal", "histogram", "date", "stock"]
    df = pd.DataFrame(arr, columns=cols)
    
    first_col = df.pop("date")
    df.insert(0, "date", first_col)
    
    df.set_index(keys='date', inplace=True)
    
    return df

final_transformation_pipe = make_pipeline(
    FunctionTransformer(final_transformation)
)

final_processing = Pipeline(
    [
        ("final_transformation", final_transformation_pipe)
    ]
)

final_processing

Pipeline(steps=[('final_transformation',
                 Pipeline(steps=[('functiontransformer',
                                  FunctionTransformer(func=<function final_transformation at 0x7fdd6c9f43a0>))]))])

In [12]:
final_pipe = Pipeline(
    [
        ("imputer", imputer),
        ("preprocessor", preprocessor),
        ("scaler", scaler),
        ("transformation", final_processing)
    ]
)

final_pipe

Pipeline(steps=[('imputer',
                 Pipeline(steps=[('imputer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('impute',
                                                                   KNNImputer(n_neighbors=10),
                                                                   ['sma25',
                                                                    'sma100',
                                                                    'sma200',
                                                                    'rsi',
                                                                    'macd',
                                                                    'signal',
                                                                    'histogram'])]))])),
                ('preprocessor',
                 Pipeline(steps=[('transformation',
                                  Pipeline(steps=[('functiontransformer',
                                                   FunctionTransformer(func=<function df_and_colu...
                                                    transformers=[('num',
                                                                   Pipeline(steps=[('scaler',
                                                                                    MinMaxScaler())]),
                                                                   ['open',
                                                                    'high',
                                                                    'low',
                                                                    'close',
                                                                    'volume',
                                                                    'vwap',
                                                                    'sma25',
                                                                    'sma100',
                                                                    'sma200',
                                                                    'rsi',
                                                                    'macd',
                                                                    'signal',
                                                                    'histogram'])]))])),
                ('transformation',
                 Pipeline(steps=[('final_transformation',
                                  Pipeline(steps=[('functiontransformer',
                                                   FunctionTransformer(func=<function final_transformation at 0x7fdd6c9f43a0>))]))]))])

In [13]:
final_pipe.fit_transform(data[0])

,open,high,low,close,volume,vwap,sma25,sma100,sma200,rsi,macd,signal,histogram,stock
date,,,,,,,,,,,,,,
2018-08-31,0.057931,0.05447,0.06204,0.058653,0.168732,0.0578,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-04,0.0629,0.073962,0.068091,0.078549,0.611726,0.073274,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-05,0.0887,0.08639,0.077176,0.081647,0.837191,0.081194,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-06,0.079918,0.077154,0.079674,0.077034,0.442654,0.077922,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-07,0.072022,0.075048,0.07689,0.073868,0.371007,0.076237,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-24,0.64452,0.641267,0.605704,0.58619,0.321712,0.60702,0.705034,0.779901,0.724236,0.239357,0.360995,0.451613,0.330443,AMD
2023-08-25,0.577161,0.590194,0.596487,0.589288,0.202084,0.595187,0.702453,0.780479,0.726297,0.250737,0.347151,0.433955,0.320592,AMD
2023-08-28,0.592852,0.589854,0.605871,0.591766,0.13457,0.599475,0.699245,0.78139,0.728588,0.260325,0.339155,0.434939,0.310832,AMD


In [14]:
dataframes = []

for df in data:
    preprocessed_df = final_pipe.fit_transform(df)
    
    dataframes.append(preprocessed_df)

In [15]:
filtered_dataframes = []

for df in dataframes:
    if df.shape == (1257, 14):
        filtered_dataframes.append(df)

In [16]:
unequal_shape = []

for df in dataframes:
    if df.shape != (1257, 14):
        unequal_shape.append(df)

In [17]:
# Check
num_dataframes = len(dataframes)
print("Number of dataframes:", num_dataframes)

unequal = len(unequal_shape)
print(f"Length of unequal dataframe: {unequal}")

f = len(filtered_dataframes)
print(f"Length of filtered dataframe: {f}")

Number of dataframes: 100
Length of unequal dataframe: 3
Length of filtered dataframe: 97


In [18]:
# Step 5: Put in tensor
# Stack the dataframes into a 3D tensor
tensor = np.stack(filtered_dataframes, axis=0)

In [19]:
# Check
# Display the shape of the resulting tensor
print("Shape of the 3D tensor:", tensor.shape)

Shape of the 3D tensor: (97, 1257, 14)


In [20]:
tensor[76]

array([[0.20004178586352095, 0.20226117327469118, 0.23400905487757118,
        ..., 0.4595276891317182, 0.5664059934268184, 'BLK'],
       [0.20399537141204083, 0.20060151348554423, 0.23066438989237548,
        ..., 0.4595276891317182, 0.5664059934268184, 'BLK'],
       [0.202291793899264, 0.20409417243386585, 0.2358027979258377, ...,
        0.4595276891317182, 0.5664059934268184, 'BLK'],
       ...,
       [0.5335894056764489, 0.5316887652803829, 0.549324017357793, ...,
        0.43506382662747445, 0.4012075341873016, 'BLK'],
       [0.5293143904085372, 0.5444990621563935, 0.5523318659627464, ...,
        0.4894521078953867, 0.40405588699495076, 'BLK'],
       [0.5504162514866123, 0.5574995149084795, 0.5731024705089843, ...,
        0.5321440419844065, 0.4110951020510112, 'BLK']], dtype=object)

In [21]:
tensor

array([[[0.057930565010211044, 0.05446957348546591,
         0.062040408367244934, ..., 0.5406818187361089,
         0.4516718198171634, 'AMD'],
        [0.0628999319264806, 0.07396155935886986, 0.06809095452273863,
         ..., 0.5406818187361089, 0.4516718198171634, 'AMD'],
        [0.08869979577944181, 0.0863902472154306, 0.07717569786535304,
         ..., 0.5406818187361089, 0.4516718198171634, 'AMD'],
        ...,
        [0.5928522804629, 0.5898539798967671, 0.6058713500392662, ...,
         0.43493919830286043, 0.3108315464853732, 'AMD'],
        [0.5835942818243702, 0.6068330616680249, 0.612265295923467, ...,
         0.4789557046605755, 0.30507654560980335, 'AMD'],
        [0.606004084411164, 0.6081234718826406, 0.6316127650460485, ...,
         0.5192661680165183, 0.30299010966967843, 'AMD']],

       [[0.5657513613717993, 0.5492346355951564, 0.5726476014760147,
         ..., 0.5391043102288354, 0.6274015212133411, 'XOM'],
        [0.5677210056772101, 0.5518620059401416, 0.5

In [22]:
x, y, z = tensor.shape

In [23]:
x, y, z

(97, 1257, 14)

In [24]:
index_level_0 = []

In [25]:
filtered_dataframes[1]

,open,high,low,close,volume,vwap,sma25,sma100,sma200,rsi,macd,signal,histogram,stock
date,,,,,,,,,,,,,,
2018-08-31,0.565751,0.549235,0.572648,0.555404,0.082809,0.558336,0.484412,0.458885,0.42104,0.523036,0.628623,0.539104,0.627402,XOM
2018-09-04,0.567721,0.551862,0.573109,0.556772,0.060987,0.561005,0.484412,0.458885,0.42104,0.523036,0.628623,0.539104,0.627402,XOM
2018-09-05,0.561812,0.560315,0.570687,0.568969,0.120408,0.567952,0.484412,0.458885,0.42104,0.523036,0.628623,0.539104,0.627402,XOM
2018-09-06,0.575831,0.562714,0.57922,0.55871,0.113811,0.565992,0.484412,0.458885,0.42104,0.523036,0.628623,0.539104,0.627402,XOM
2018-09-07,0.564709,0.566027,0.573224,0.574327,0.150455,0.573752,0.484412,0.458885,0.42104,0.523036,0.628623,0.539104,0.627402,XOM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-24,0.869077,0.861149,0.875923,0.853853,0.083093,0.864061,0.914488,0.955671,0.998176,0.456541,0.66255,0.42685,0.703305,XOM
2023-08-25,0.880779,0.874457,0.882265,0.875513,0.116869,0.877113,0.915837,0.954747,0.997756,0.541385,0.660224,0.443434,0.695056,XOM
2023-08-28,0.892596,0.887137,0.903252,0.885887,0.079049,0.891377,0.917555,0.953678,0.997775,0.578324,0.664314,0.473236,0.689359,XOM


In [26]:
filtered_dataframes

[                open      high       low     close    volume      vwap  \
 date                                                                     
 2018-08-31  0.057931   0.05447   0.06204  0.058653  0.168732    0.0578   
 2018-09-04    0.0629  0.073962  0.068091  0.078549  0.611726  0.073274   
 2018-09-05    0.0887   0.08639  0.077176  0.081647  0.837191  0.081194   
 2018-09-06  0.079918  0.077154  0.079674  0.077034  0.442654  0.077922   
 2018-09-07  0.072022  0.075048   0.07689  0.073868  0.371007  0.076237   
 ...              ...       ...       ...       ...       ...       ...   
 2023-08-24   0.64452  0.641267  0.605704   0.58619  0.321712   0.60702   
 2023-08-25  0.577161  0.590194  0.596487  0.589288  0.202084  0.595187   
 2023-08-28  0.592852  0.589854  0.605871  0.591766   0.13457  0.599475   
 2023-08-29  0.583594  0.606833  0.612265  0.614553  0.118787  0.619034   
 2023-08-30  0.606004  0.608123  0.631613  0.618753  0.033785  0.623717   
 
                sma25  

In [27]:
filtered_dataframes[0]

,open,high,low,close,volume,vwap,sma25,sma100,sma200,rsi,macd,signal,histogram,stock
date,,,,,,,,,,,,,,
2018-08-31,0.057931,0.05447,0.06204,0.058653,0.168732,0.0578,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-04,0.0629,0.073962,0.068091,0.078549,0.611726,0.073274,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-05,0.0887,0.08639,0.077176,0.081647,0.837191,0.081194,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-06,0.079918,0.077154,0.079674,0.077034,0.442654,0.077922,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-07,0.072022,0.075048,0.07689,0.073868,0.371007,0.076237,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-24,0.64452,0.641267,0.605704,0.58619,0.321712,0.60702,0.705034,0.779901,0.724236,0.239357,0.360995,0.451613,0.330443,AMD
2023-08-25,0.577161,0.590194,0.596487,0.589288,0.202084,0.595187,0.702453,0.780479,0.726297,0.250737,0.347151,0.433955,0.320592,AMD
2023-08-28,0.592852,0.589854,0.605871,0.591766,0.13457,0.599475,0.699245,0.78139,0.728588,0.260325,0.339155,0.434939,0.310832,AMD


In [31]:
def transform_dataframe(df):
    return df.pivot(columns='stock').swaplevel(0, 1, axis=1).sort_index(axis=1)

# Apply the transformation to each data frame
transformed_dataframes = [transform_dataframe(df) for df in filtered_dataframes]

# Merge all transformed data frames into one
merged_df = pd.concat(transformed_dataframes, axis=1)

print(merged_df)

stock            AMD                                                    \
               close      high histogram       low      macd      open   
date                                                                     
2018-08-31  0.058653   0.05447  0.451672   0.06204  0.491663  0.057931   
2018-09-04  0.078549  0.073962  0.451672  0.068091  0.491663    0.0629   
2018-09-05  0.081647   0.08639  0.451672  0.077176  0.491663    0.0887   
2018-09-06  0.077034  0.077154  0.451672  0.079674  0.491663  0.079918   
2018-09-07  0.073868  0.075048  0.451672   0.07689  0.491663  0.072022   
...              ...       ...       ...       ...       ...       ...   
2023-08-24   0.58619  0.641267  0.330443  0.605704  0.360995   0.64452   
2023-08-25  0.589288  0.590194  0.320592  0.596487  0.347151  0.577161   
2023-08-28  0.591766  0.589854  0.310832  0.605871  0.339155  0.592852   
2023-08-29  0.614553  0.606833  0.305077  0.612265  0.347888  0.583594   
2023-08-30  0.618753  0.608123   0.302

In [40]:
merged_df

stock            AMD                                                    \
               close      high histogram       low      macd      open   
date                                                                     
2018-08-31  0.058653   0.05447  0.451672   0.06204  0.491663  0.057931   
2018-09-04  0.078549  0.073962  0.451672  0.068091  0.491663    0.0629   
2018-09-05  0.081647   0.08639  0.451672  0.077176  0.491663    0.0887   
2018-09-06  0.077034  0.077154  0.451672  0.079674  0.491663  0.079918   
2018-09-07  0.073868  0.075048  0.451672   0.07689  0.491663  0.072022   
...              ...       ...       ...       ...       ...       ...   
2023-08-24   0.58619  0.641267  0.330443  0.605704  0.360995   0.64452   
2023-08-25  0.589288  0.590194  0.320592  0.596487  0.347151  0.577161   
2023-08-28  0.591766  0.589854  0.310832  0.605871  0.339155  0.592852   
2023-08-29  0.614553  0.606833  0.305077  0.612265  0.347888  0.583594   
2023-08-30  0.618753  0.608123   0.30299  0.631613  0.358592  0.606004   

stock                                                                   \
                 rsi    signal    sma100    sma200     sma25    volume   
date                                                                     
2018-08-31  0.455593  0.540682  0.465189  0.525993  0.409796  0.168732   
2018-09-04  0.455593  0.540682  0.465189  0.525993  0.409796  0.611726   
2018-09-05  0.455593  0.540682  0.465189  0.525993  0.409796  0.837191   
2018-09-06  0.455593  0.540682  0.465189  0.525993  0.409796  0.442654   
2018-09-07  0.455593  0.540682  0.465189  0.525993  0.409796  0.371007   
...              ...       ...       ...       ...       ...       ...   
2023-08-24  0.239357  0.451613  0.779901  0.724236  0.705034  0.321712   
2023-08-25  0.250737  0.433955  0.780479  0.726297  0.702453  0.202084   
2023-08-28  0.260325  0.434939   0.78139  0.728588  0.699245   0.13457   
2023-08-29  0.345629  0.478956  0.782609  0.730598  0.697958  0.118787   
2023-08-30  0.360724  0.519266   0.78361  0.732431  0.696547  0.033785   

stock                      XOM                                          \
                vwap     close      high histogram       low      macd   
date                                                                     
2018-08-31    0.0578  0.555404  0.549235  0.627402  0.572648  0.628623   
2018-09-04  0.073274  0.556772  0.551862  0.627402  0.573109  0.628623   
2018-09-05  0.081194  0.568969  0.560315  0.627402  0.570687  0.628623   
2018-09-06  0.077922   0.55871  0.562714  0.627402   0.57922  0.628623   
2018-09-07  0.076237  0.574327  0.566027  0.627402  0.573224  0.628623   
...              ...       ...       ...       ...       ...       ...   
2023-08-24   0.60702  0.853853  0.861149  0.703305  0.875923   0.66255   
2023-08-25  0.595187  0.875513  0.874457  0.695056  0.882265  0.660224   
2023-08-28  0.599475  0.885887  0.887137  0.689359  0.903252  0.664314   
2023-08-29  0.619034  0.893297  0.886909    0.6864  0.903021  0.671552   
2023-08-30  0.623717  0.906692  0.897932  0.686972   0.92101  0.684873   

stock                                                                   \
                open       rsi    signal    sma100    sma200     sma25   
date                                                                     
2018-08-31  0.565751  0.523036  0.539104  0.458885   0.42104  0.484412   
2018-09-04  0.567721  0.523036  0.539104  0.458885   0.42104  0.484412   
2018-09-05  0.561812  0.523036  0.539104  0.458885   0.42104  0.484412   
2018-09-06  0.575831  0.523036  0.539104  0.458885   0.42104  0.484412   
2018-09-07  0.564709  0.523036  0.539104  0.458885   0.42104  0.484412   
...              ...       ...       ...       ...       ...       ...   
2023-08-24  0.869077  0.456541   0.42685  0.955671  0.998176  0.914488   
2023-08-25  0.880779  0.541385  0.443434  0.954747  0.997756  0.915837   
2023-08-28  0.892596  0.578324  0.473236  0.953678  0.997775  0.917555   
2

In [32]:
pd.set_option('display.max_columns', None)

In [33]:
merged_df.index = pd.to_datetime(merged_df.index)
